In [2]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import tqdm.notebook as tqdm
from plotly import graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

First we compute some auxiliary dictionaries. To help mapping names



In [8]:
output = "ENTER-HERE-OUTPUT-FILE.csv" #

In [7]:
df_stations = pd.read_json("./stations.json")


for k  in df_stations["list"][0].keys():
    
    df_stations[k] = df_stations["list"].apply( lambda x : x[k] )
df_stations = df_stations.drop( ["list"] , axis = 1 )

#df_stations = df_stations[ df_stations["description"] != "Hakaniemen kauppahalli 1" ]
serial2name = dict()
name2pos = dict( )
for x , y  in df_stations.iterrows():
    serial2name[ y["serial"]] = y["description"]
    name2pos[ y["description"] ] = ( y["latitude"] , y["longitude"] )
name2serial = {v: k for k, v in serial2name.items()}

The function build_graph takes the data frame retrieved from the API and builds a graph, based on the occurences of unique hashes. We keep track of each unique hash and count if the hash is seen by other station at a future time.

In [5]:
def build_graph( df_ ):
    graph = nx.MultiDiGraph() 
    #graph.add_nodes_from(   [  serial2name[x] for x in  df_["serial"].unique() ]  )
    for  i , x in  tqdm.tqdm( df_.iterrows() ) :

        node1 = x[ "serial" ]
        name1 = serial2name[ node1 ]
        hash1 = x["hash"]
        next_data = df[ i: ]
        next_data = next_data[ next_data["time"] != x["time"] ]   #
        next_data = next_data[ next_data["hash"] == hash1 ]
        next_data = next_data[ next_data["serial"] != node1 ]
        if ( len( next_data) != 0 ):
            next_stations = next_data["serial"]
            edges = []
            for s in next_stations:
                name2 = serial2name[s]
                edges.append(   ( name1  , name2 ) )
            #print( edges )
            graph.add_edges_from(  edges )
            
    return graph

In [ ]:
graphs = []
for f in files[:]:

    df = ... # logic to read the dataframe
    df = df.drop_duplicates( subset =["hash" , "serial"] )
    g = build_graph( df )
    graphs.append( g )

In [ ]:
total_edges = []
for g in graphs:
    total_edges += g.edges
    
total_graph = nx.MultiDiGraph()
total_graph.add_edges_from( total_edges )


In [ ]:
A = nx.adjacency_matrix( total_graph, nodelist=total_graph.nodes ).todense()#.shape
nodes = total_graph.nodes
df_A = pd.DataFrame( A )
df_A.columns = [  name2serial[ x] for x in nodes ]  
df_A.index =  [  name2serial[ x] for x in nodes ]
df_a.to_csv(output , index = False )